In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset,TensorDataset
from autoencoder import Autoencoder
import torchvision
from model2 import classification_model
import copy
import partition
from pca import PCADigitReducer
from autoencoder import reduce_dimensions
from training import train,test
from federated_learning import distribute_global_model, federated_averaging

# Preprocessing

In [2]:
# Predefined stuff

n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10
num_clusters = 2

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [3]:
batch_size_train = 64
batch_size_test = 1000

fashion_mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3204,))  
])

fashion_mnist_train_loader = DataLoader(
    datasets.FashionMNIST('/files/', train=True, download=True, transform=fashion_mnist_transform),
    batch_size=batch_size_train, shuffle=True
)

fashion_mnist_test_loader = DataLoader(
    datasets.FashionMNIST('/files/', train=False, download=True, transform=fashion_mnist_transform),
    batch_size=batch_size_test, shuffle=True
)

100%|█████████████████████████████████████████████████████████████████████████████| 26.4M/26.4M [00:00<00:00, 29.5MB/s]


Extracting /files/FashionMNIST\raw\train-images-idx3-ubyte.gz to /files/FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 3.12MB/s]

Extracting /files/FashionMNIST\raw\train-labels-idx1-ubyte.gz to /files/FashionMNIST\raw




100%|█████████████████████████████████████████████████████████████████████████████| 4.42M/4.42M [00:00<00:00, 18.7MB/s]


Extracting /files/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to /files/FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████████████| 5.15k/5.15k [00:00<?, ?B/s]

Extracting /files/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to /files/FashionMNIST\raw



In [4]:
train_loader_pca = copy.copy(fashion_mnist_train_loader)
test_loader_pca = copy.copy(fashion_mnist_test_loader)

train_loader_auto = copy.copy(fashion_mnist_train_loader)
test_loader_auto = copy.copy(fashion_mnist_test_loader)

In [5]:
class CustomTensorDataset(TensorDataset):
    def __init__(self, *tensors):
        super().__init__(*tensors)
        self.data = tensors[0]
        self.targets = tensors[1] 

In [6]:
train_data = []
train_labels = []
for data, labels in train_loader_pca:
    train_data.append(data.view(data.size(0), -1))  
    train_labels.append(labels)
train_data = torch.cat(train_data, dim=0)  
train_labels = torch.cat(train_labels, dim=0)

train_data_np = train_data.numpy()

pca = PCADigitReducer(100)
train_data_reduced = pca.fit_transform(train_data_np)  

train_data_reconstructed_np = pca.inverse_transform(train_data_reduced) 
train_data_reconstructed = torch.tensor(train_data_reconstructed_np, dtype=torch.float32)

train_data_reconstructed = train_data_reconstructed.view(-1, 1, 28, 28)

train_data_reconstructed = (train_data_reconstructed - 0.2860) / 0.3204

batch_size_train = train_loader_pca.batch_size
train_dataset_pca = CustomTensorDataset(train_data_reconstructed, train_labels)
train_loader_reduced_pca = DataLoader(train_dataset_pca, batch_size=batch_size_train, shuffle=True)

In [7]:
trainingset_pca = train_loader_reduced_pca.dataset

print(trainingset_pca.targets)

tensor([5, 4, 9,  ..., 1, 9, 2])


In [8]:
# Autoencoder
latent_dim = 100  
autoencoder = Autoencoder(latent_dim=latent_dim)
auto_criterion = nn.MSELoss()
auto_optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
auto_num_epochs = 5
for epoch in range(auto_num_epochs): 
    for images, _ in train_loader_auto:
        auto_optimizer.zero_grad()
        reconstructed = autoencoder(images)
        loss = auto_criterion(reconstructed, images)  
        loss.backward()
        auto_optimizer.step()
        
    print(f"Epoch [{epoch+1}/5], Loss: {loss.item()}")

Epoch [1/5], Loss: 0.6312184929847717
Epoch [2/5], Loss: 0.6559471487998962
Epoch [3/5], Loss: 0.6077186465263367
Epoch [4/5], Loss: 0.581427276134491
Epoch [5/5], Loss: 0.6195714473724365


In [9]:
autoencoder.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
latent_features, labels = reduce_dimensions(train_loader_auto, autoencoder.encoder, device)
latent_features = latent_features.detach()

reconstructed_images = autoencoder.decoder(latent_features.to(device))  
reconstructed_images = reconstructed_images.view(-1, 1, 28, 28)  # Reshape to [batch_size, channels, height, width]

reconstructed_dataset = CustomTensorDataset(reconstructed_images.cpu(), labels)  
reduced_train_loader_auto = DataLoader(reconstructed_dataset, batch_size=batch_size_train, shuffle=True)

In [10]:
# Now partition them into 4 clients for federated learning
# pca 4 clients
trainingset_pca = train_loader_reduced_pca.dataset
partitioned_data_pca = partition.balanced_dirichlet_partition(trainingset_pca, partitions_number=4, alpha=0.5)
# Check sizes

for i in partitioned_data_pca:
    print(len(partitioned_data_pca[i]))

23974
7961
15584
12481


In [13]:
# Apply clustering
import cluster
cluster = cluster.Cluster(num_clusters=num_clusters)

targets = trainingset_pca.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_pca, targets, num_classes)

partitioned_data_pca = clustered_data

for key, value in partitioned_data_pca.items():
    print(f"Key: {key}, Length of Values: {len(value)}")

Key: 0, Length of Values: 23974
Key: 1, Length of Values: 36026


In [17]:
pca_client_loaders = [
    DataLoader(Subset(trainingset_pca, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_pca.values()
]

In [18]:
# check shape
x = pca_client_loaders[0].dataset
for i in range(len(x)):
    sample, label = x[i]
    print(f"Sample {i} shape: {sample.shape}, Label: {label}")

Sample 0 shape: torch.Size([1, 28, 28]), Label: 4
Sample 1 shape: torch.Size([1, 28, 28]), Label: 0
Sample 2 shape: torch.Size([1, 28, 28]), Label: 8
Sample 3 shape: torch.Size([1, 28, 28]), Label: 4
Sample 4 shape: torch.Size([1, 28, 28]), Label: 4
Sample 5 shape: torch.Size([1, 28, 28]), Label: 2
Sample 6 shape: torch.Size([1, 28, 28]), Label: 0
Sample 7 shape: torch.Size([1, 28, 28]), Label: 1
Sample 8 shape: torch.Size([1, 28, 28]), Label: 0
Sample 9 shape: torch.Size([1, 28, 28]), Label: 4
Sample 10 shape: torch.Size([1, 28, 28]), Label: 2
Sample 11 shape: torch.Size([1, 28, 28]), Label: 8
Sample 12 shape: torch.Size([1, 28, 28]), Label: 2
Sample 13 shape: torch.Size([1, 28, 28]), Label: 4
Sample 14 shape: torch.Size([1, 28, 28]), Label: 0
Sample 15 shape: torch.Size([1, 28, 28]), Label: 0
Sample 16 shape: torch.Size([1, 28, 28]), Label: 2
Sample 17 shape: torch.Size([1, 28, 28]), Label: 8
Sample 18 shape: torch.Size([1, 28, 28]), Label: 4
Sample 19 shape: torch.Size([1, 28, 28]),

In [19]:
# classic
trainingset = fashion_mnist_train_loader.dataset
partitioned_data_classic = partition.balanced_dirichlet_partition(trainingset, partitions_number=4, alpha=0.5)

for i in partitioned_data_classic:
    print(len(partitioned_data_classic[i]))

23974
7961
15584
12481


In [22]:
# Apply clustering
import cluster
cluster = cluster.Cluster(num_clusters=num_clusters)

targets = trainingset.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_classic, targets, num_classes)

partitioned_data_classic = clustered_data

for key, value in partitioned_data_classic.items():
    print(f"Key: {key}, Length of Values: {len(value)}")

Key: 0, Length of Values: 23974
Key: 1, Length of Values: 36026


In [23]:
classic_client_loaders = [
    DataLoader(Subset(trainingset, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_classic.values()
]

In [24]:
# auto 4 clients
trainingset_auto = reduced_train_loader_auto.dataset
print(trainingset_auto.targets)
print(trainingset_auto.data.shape)
partitioned_data_auto = partition.balanced_dirichlet_partition(trainingset_auto, partitions_number=4, alpha=0.5)
# Check sizes

for i in partitioned_data_auto:
    print(len(partitioned_data_auto[i]))

tensor([6, 3, 2,  ..., 0, 6, 8])
torch.Size([60000, 1, 28, 28])
23974
7961
15584
12481


In [25]:
# Apply clustering
import cluster
cluster = cluster.Cluster(num_clusters=num_clusters)

targets = trainingset_auto.targets
num_classes = len(set(targets)) 
clustered_data = cluster.apply_clustering(partitioned_data_auto, targets, num_classes)

partitioned_data_auto = clustered_data

for key, value in partitioned_data_auto.items():
    print(f"Key: {key}, Length of Values: {len(value)}")

Key: 0, Length of Values: 23974
Key: 1, Length of Values: 36026


In [26]:
auto_client_loaders = [
    DataLoader(Subset(trainingset_auto, indices), batch_size=batch_size_train, shuffle=True)
    for indices in partitioned_data_auto.values()
]

In [31]:
# defining model for pca and autoencoder

global_model_pca = classification_model()
global_model_auto = classification_model()
gloabl_model_classic =classification_model()

num_clients = 2
# classic models
local_models_classic = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# pca models 
local_models_pca = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]
# autoencodere models
local_model_autoencoder = [copy.deepcopy(global_model_pca) for _ in range(num_clients)]

# Classic

In [33]:
rounds_classic = 4

for round_idx in range(rounds_classic):
    
    print(f"Round {round_idx + 1}/{rounds_classic}")

    local_weights_classic = []
    for client_idx, client_model in enumerate(local_models_classic):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, classic_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_classic.append(client_weights)
        
    print(f"after training{local_models_classic}")
    global_weights_classic = federated_averaging(local_weights_classic)
    print(f"after fedaveraging{local_models_classic}")

    distribute_global_model(global_weights_classic,local_models_classic,single=False)

    distribute_global_model(global_weights_classic,gloabl_model_classic,single=True)
    test_losses = []
    test(gloabl_model_classic,fashion_mnist_test_loader,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.306991
Train Epoch: 1 [640/23974 (3%)]	Loss: 2.269633
Train Epoch: 1 [1280/23974 (5%)]	Loss: 2.255831
Train Epoch: 1 [1920/23974 (8%)]	Loss: 2.172280
Train Epoch: 1 [2560/23974 (11%)]	Loss: 2.124580
Train Epoch: 1 [3200/23974 (13%)]	Loss: 2.110742
Train Epoch: 1 [3840/23974 (16%)]	Loss: 2.049162
Train Epoch: 1 [4480/23974 (19%)]	Loss: 2.011262
Train Epoch: 1 [5120/23974 (21%)]	Loss: 1.975160
Train Epoch: 1 [5760/23974 (24%)]	Loss: 1.883742
Train Epoch: 1 [6400/23974 (27%)]	Loss: 1.791696
Train Epoch: 1 [7040/23974 (29%)]	Loss: 1.724840
Train Epoch: 1 [7680/23974 (32%)]	Loss: 1.583394
Train Epoch: 1 [8320/23974 (35%)]	Loss: 1.465614
Train Epoch: 1 [8960/23974 (37%)]	Loss: 1.478463
Train Epoch: 1 [9600/23974 (40%)]	Loss: 1.344192
Train Epoch: 1 [10240/23974 (43%)]	Loss: 1.646999
Train Epoch: 1 [10880/23974 (45%)]	Loss: 1.424014
Train Epoch: 1 [11520/23974 (48%)]	Loss: 1.302622
Train Epoch: 1 [12160/23974 (51%)]	Loss: 1.370

C:\Users\nazek\anaconda3\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.6581, Accuracy: 7609/10000 (76%)

Round 2/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 0.625039
Train Epoch: 1 [640/23974 (3%)]	Loss: 0.576580
Train Epoch: 1 [1280/23974 (5%)]	Loss: 0.778853
Train Epoch: 1 [1920/23974 (8%)]	Loss: 0.647875
Train Epoch: 1 [2560/23974 (11%)]	Loss: 0.790537
Train Epoch: 1 [3200/23974 (13%)]	Loss: 0.691498
Train Epoch: 1 [3840/23974 (16%)]	Loss: 0.747156
Train Epoch: 1 [4480/23974 (19%)]	Loss: 0.635502
Train Epoch: 1 [5120/23974 (21%)]	Loss: 0.841264
Train Epoch: 1 [5760/23974 (24%)]	Loss: 0.659861
Train Epoch: 1 [6400/23974 (27%)]	Loss: 0.626645
Train Epoch: 1 [7040/23974 (29%)]	Loss: 0.460800
Train Epoch: 1 [7680/23974 (32%)]	Loss: 0.452291
Train Epoch: 1 [8320/23974 (35%)]	Loss: 0.546879
Train Epoch: 1 [8960/23974 (37%)]	Loss: 0.714268
Train Epoch: 1 [9600/23974 (40%)]	Loss: 0.522032
Train Epoch: 1 [10240/23974 (43%)]	Loss: 0.890465
Train Epoch: 1 [10880/23974 (45%)]	Loss: 0.697515
Train Epoch: 1 [11520/23974 (48%)]	Los

# PCA

In [34]:
rounds_pca = 4

for round_idx in range(rounds_pca):
    
    print(f"Round {round_idx + 1}/{rounds_pca}")

    local_weights_pca = []
    for client_idx, client_model in enumerate(local_models_pca):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)

        train_losses = []
        train_counter = []


        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, pca_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_pca.append(client_weights)
        
    print(f"after training{local_models_pca}")
    global_weights_pca = federated_averaging(local_weights_pca)
    print(f"after fedaveraging{local_models_pca}")

    distribute_global_model(global_weights_pca,local_models_pca,single=False)

    distribute_global_model(global_weights_pca,global_model_pca,single=True)
    test_losses = []
    test(global_model_pca,test_loader_pca,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.398217
Train Epoch: 1 [640/23974 (3%)]	Loss: 2.161974
Train Epoch: 1 [1280/23974 (5%)]	Loss: 2.022100
Train Epoch: 1 [1920/23974 (8%)]	Loss: 2.066435
Train Epoch: 1 [2560/23974 (11%)]	Loss: 1.732679
Train Epoch: 1 [3200/23974 (13%)]	Loss: 1.763144
Train Epoch: 1 [3840/23974 (16%)]	Loss: 1.402403
Train Epoch: 1 [4480/23974 (19%)]	Loss: 1.692639
Train Epoch: 1 [5120/23974 (21%)]	Loss: 1.448067
Train Epoch: 1 [5760/23974 (24%)]	Loss: 1.535750
Train Epoch: 1 [6400/23974 (27%)]	Loss: 1.279408
Train Epoch: 1 [7040/23974 (29%)]	Loss: 1.292679
Train Epoch: 1 [7680/23974 (32%)]	Loss: 0.886002
Train Epoch: 1 [8320/23974 (35%)]	Loss: 1.204514
Train Epoch: 1 [8960/23974 (37%)]	Loss: 1.010436
Train Epoch: 1 [9600/23974 (40%)]	Loss: 0.865043
Train Epoch: 1 [10240/23974 (43%)]	Loss: 0.929127
Train Epoch: 1 [10880/23974 (45%)]	Loss: 0.966749
Train Epoch: 1 [11520/23974 (48%)]	Loss: 1.119513
Train Epoch: 1 [12160/23974 (51%)]	Loss: 0.933

In [35]:
# Check shapes for autoencoder

data_test = auto_client_loaders[0].dataset
for i in range(len(data_test)):
    sample, label = data_test[i]
    print(f"Sample {i} shape: {sample.shape}, Label: {label}")

Sample 0 shape: torch.Size([1, 28, 28]), Label: 4
Sample 1 shape: torch.Size([1, 28, 28]), Label: 0
Sample 2 shape: torch.Size([1, 28, 28]), Label: 8
Sample 3 shape: torch.Size([1, 28, 28]), Label: 4
Sample 4 shape: torch.Size([1, 28, 28]), Label: 4
Sample 5 shape: torch.Size([1, 28, 28]), Label: 2
Sample 6 shape: torch.Size([1, 28, 28]), Label: 0
Sample 7 shape: torch.Size([1, 28, 28]), Label: 1
Sample 8 shape: torch.Size([1, 28, 28]), Label: 0
Sample 9 shape: torch.Size([1, 28, 28]), Label: 4
Sample 10 shape: torch.Size([1, 28, 28]), Label: 2
Sample 11 shape: torch.Size([1, 28, 28]), Label: 8
Sample 12 shape: torch.Size([1, 28, 28]), Label: 2
Sample 13 shape: torch.Size([1, 28, 28]), Label: 4
Sample 14 shape: torch.Size([1, 28, 28]), Label: 0
Sample 15 shape: torch.Size([1, 28, 28]), Label: 0
Sample 16 shape: torch.Size([1, 28, 28]), Label: 2
Sample 17 shape: torch.Size([1, 28, 28]), Label: 8
Sample 18 shape: torch.Size([1, 28, 28]), Label: 4
Sample 19 shape: torch.Size([1, 28, 28]),

# Autoencoder

In [36]:
rounds_auto = 4

for round_idx in range(rounds_auto):
    print(f"Round {round_idx + 1}/{rounds_auto}")

    local_weights_auto = []
    for client_idx, client_model in enumerate(local_model_autoencoder):
        print(f"Training client {client_idx + 1}")
        
        optimizer = optim.SGD(client_model.parameters(), lr=learning_rate,
                      momentum=momentum)
        
        train_losses = []
        train_counter = []

        for epoch in range(1, n_epochs + 1):  
            train(epoch, client_model, auto_client_loaders[client_idx], optimizer, log_interval, train_losses, train_counter)
        
        client_weights = [param.data.numpy() for param in client_model.parameters()]
        local_weights_auto.append(client_weights)
        
    global_weights_auto = federated_averaging(local_weights_auto)

    distribute_global_model(global_weights_auto,local_model_autoencoder,single=False)

    distribute_global_model(global_weights_auto, global_model_auto,single=True)
    test_losses = []
    test(global_model_auto,test_loader_auto,test_losses)

Round 1/4
Training client 1
Train Epoch: 1 [0/23974 (0%)]	Loss: 2.291927
Train Epoch: 1 [640/23974 (3%)]	Loss: 2.258871
Train Epoch: 1 [1280/23974 (5%)]	Loss: 2.252685
Train Epoch: 1 [1920/23974 (8%)]	Loss: 2.235671
Train Epoch: 1 [2560/23974 (11%)]	Loss: 2.205263
Train Epoch: 1 [3200/23974 (13%)]	Loss: 2.155766
Train Epoch: 1 [3840/23974 (16%)]	Loss: 2.206106
Train Epoch: 1 [4480/23974 (19%)]	Loss: 2.069680
Train Epoch: 1 [5120/23974 (21%)]	Loss: 2.036014
Train Epoch: 1 [5760/23974 (24%)]	Loss: 2.062258
Train Epoch: 1 [6400/23974 (27%)]	Loss: 1.993488
Train Epoch: 1 [7040/23974 (29%)]	Loss: 2.067911
Train Epoch: 1 [7680/23974 (32%)]	Loss: 1.978579
Train Epoch: 1 [8320/23974 (35%)]	Loss: 2.058172
Train Epoch: 1 [8960/23974 (37%)]	Loss: 2.095334
Train Epoch: 1 [9600/23974 (40%)]	Loss: 1.845769
Train Epoch: 1 [10240/23974 (43%)]	Loss: 1.950100
Train Epoch: 1 [10880/23974 (45%)]	Loss: 1.822892
Train Epoch: 1 [11520/23974 (48%)]	Loss: 1.682884
Train Epoch: 1 [12160/23974 (51%)]	Loss: 1.813